In [1]:
from neo4j import GraphDatabase
import threading
import time

<hr>

In [2]:
# Параметри з'єднання з Neo4j
uri = "bolt://localhost:7687"
user = "neo4j"
password = "UIOP1234"

<hr>

In [3]:
# Ініціалізація поля "likes"
def initialize_field(driver):
    with driver.session() as session:
        session.run("MATCH (i:Item {id: 1}) \
                     SET i.likes = 0")

In [4]:
# Інкремент лічильника на '+1'
def increment_likes(driver):
    with driver.session() as session:
        for _ in range(10000):
            session.run("MATCH (i:Item {id: 1}) \
                         SET i.likes = i.likes + 1")

<hr>

In [5]:
if __name__ == "__main__":
    driver = GraphDatabase.driver(uri, auth=(user, password))
    initialize_field(driver)

    print(f"\n{'-' * 50}\nExecuting task in Neo4j...\n{'-' * 50}")
    start_time = time.time()

    threads = [threading.Thread(target=increment_likes, args=(driver,)) for _ in range(10)]

    for thread in threads: thread.start()
    for thread in threads: thread.join()

    end_time = time.time()
    print(f"Task executed in: {end_time - start_time:.2f} seconds")

    # Перевірка фінального значення
    with driver.session() as session:
        result = session.run("MATCH (i:Item {id: 1}) RETURN i.likes AS likes")
        for record in result:
            print(f"Final counter value (Neo4j): {record['likes']}")

    print(f"{'-' * 50}\n")
    driver.close()


--------------------------------------------------
Executing task in Neo4j...
--------------------------------------------------
Task executed in: 2781.56 seconds
Final counter value (Neo4j): 100000
--------------------------------------------------

